Use concat as a fast replacement of roll

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [2]:
def upwind_tendency_vect(c, u, dx, dt):
    '''
    Vectorized version of advection tendency with periodic boundary

    Args:
      c: 1d numpy array, density field
      u: 1d numpy array, wind field
      dx: float, grid spacing (assume uniform)
      dt: float, time step

    Returns:
      1d numpy array with same shape as `c`
    '''

    # re-stagger to C-grid, pointing from box[i-1] to box[i]
    # remove this step if input wind is already staggered
    # u = 0.5*(u+np.roll(u, 1))

    # c_l = np.roll(c, 1)  # so c_l[i] == c[i-1], with periodic boundary
    c_l = np.concatenate((c[-1:], c[:-1]))
            
    flux_right = np.maximum(u, 0) * c_l
    flux_left = np.minimum(u, 0) * c
    flux = flux_right + flux_left
            
    # tendency = flux - np.roll(flux, -1)
    tendency = flux - np.concatenate((flux[1:], flux[:1]))
    
    return tendency*dt/dx

In [3]:
nx = 100
Lx = 1
dx = Lx/nx
dt = 0.01
u = np.ones(nx) * -0.5
c0 = np.zeros(nx)
c0[int(nx*0.2):int(nx*0.4)] = 1.0

In [4]:
%%time
nt = 10000

c1 = c0.copy()
for _ in range(nt):
    c1 += upwind_tendency_vect(c1, u, dx, dt)
# 3x faster than np.roll; still 2~3x slower than numba

CPU times: user 129 ms, sys: 8.7 ms, total: 137 ms
Wall time: 133 ms


# TF Eager

In [5]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()

In [6]:
def upwind_tendency_tf(c, u, dx, dt):
    '''
    Tensorflow vectorized version of advection tendency with periodic boundary

    Args:
      c: 1d Tensor, density field
      u: 1d Tensor, wind field
      dx: 0d Tensor, grid spacing (assume uniform)
      dt: 0d Tensor, time step

    Returns:
      1d Tensor with same shape as `c`
    '''
    
    # re-stagger to C-grid, pointing from box[i-1] to box[i]
    # remove this step if input wind is already staggered
    # u = 0.5*(u+tf.manip.roll(u, 1, 0))

    # c_l = tf.manip.roll(c, 1, 0)  # so c_l[i] == c[i-1], with periodic boundary
    c_l = tf.concat((c[-1:], c[:-1]), axis=0)
    
    # https://www.tensorflow.org/tutorials/non-ml/pdes
            
    flux_right = tf.maximum(u, 0) * c_l
    flux_left = tf.minimum(u, 0) * c
    flux = flux_right + flux_left
            
    tendency = flux - tf.concat((flux[1:], flux[:1]), axis=0)
    
    return tendency*dt/dx

In [7]:
%%time
nt = 10000
u_tensor = tf.convert_to_tensor(u)
dx_tensor = tf.convert_to_tensor(dx, dtype=tf.float64)
dt_tensor = tf.convert_to_tensor(dt, dtype=tf.float64)
c_tensor = tf.convert_to_tensor(c0)

for _ in range(nt):
    c_tensor += upwind_tendency_tf(c_tensor, u_tensor, dx_tensor, dt_tensor)
    
# Even slower than using tf.manip.roll()?

CPU times: user 3.55 s, sys: 29.9 ms, total: 3.58 s
Wall time: 3.62 s


In [8]:
np.array_equal(c_tensor.numpy(), c1)

True